##### Copyright 2023 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/function_calling_python_quickstart"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/function_calling_python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/function_calling_python_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

You can provide Gemini models with descriptions of functions, the model may ask you to call a function and send back the result to help the model handle your query.

## Setup


### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:


In [2]:
#!pip install -U google-generativeai

### Import packages

Import the necessary packages.

In [3]:
import pathlib
import textwrap

import google.generativeai as genai


from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Set up your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [4]:

try:
    # Used to securely store your API key
    from google.colab import userdata
    
    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
    
genai.configure(api_key=GOOGLE_API_KEY)

## Function Basics

You can pass a list of functions to the `tools` argument when creating a `genai.GenerativeModel`.

> Important: The function arguments require type annotations for this to work. A limited selection of argument types are allowed.

In [13]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[multiply])

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x10a5e3890>,
)

The recomended way to use function calling is through the chat interface. The main reason is that `FunctionCalls` fit nicely into chat's multi-turn structure.

In [14]:
chat = model.start_chat(enable_automatic_function_calling=True)

With automatic function calling enabled `chat.send_message` automatically calls your function if the model asks it to.

It appears to simply return a text response, containing the correct answer:

In [15]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'That is 2508 mittens in total.'

In [16]:
57*44

2508

If you look in the `ChatSession.history` you can see the sequence of events:

1. You sent the question.
2. The model replied with a `glm.FunctionCall`.
3. The `genai.ChatSession` executed the function locally and sent the model back a `glm.FunctionResponse`.
4. The model used the function output in its answer.

Note that the model can potentially respond with multiple function calls before returning a text response.

In [17]:
for content in chat.history:
    print(content.role, "->", content.parts[0])
    print('-'*80+'\n')

user -> text: "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"

--------------------------------------------------------------------------------

model -> function_call {
  name: "multiply"
  args {
    fields {
      key: "b"
      value {
        number_value: 44
      }
    }
    fields {
      key: "a"
      value {
        number_value: 57
      }
    }
  }
}

--------------------------------------------------------------------------------

user -> function_response {
  name: "multiply"
  response {
    fields {
      key: "result"
      value {
        number_value: 2508
      }
    }
  }
}

--------------------------------------------------------------------------------

model -> text: "That is 2508 mittens in total."

--------------------------------------------------------------------------------



If you need more control, you can do any of this manually using chat without automatic function enabled, or using `GenerativeModel.generate_content`. 

## Examples

### Structured data extraction

While the Gemini API doesn't currently have a json mode 

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro')

story = model.generate_content("Tell me a story about a girl with magic backpack and her family").text

In [ ]:
print(story)

## Low level access

The `google.ai.generativelanguage` client library provides access to the low level types required for function calling.

In [ ]:
import google.ai.generativelanguage as glm

If you peek inside the model's `tools` attribute, you can see how it describes the function(s) you passed it to the model:

In [ ]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                             tools=[multiply])

model._tools.to_proto()

This is a list of `glm.Tool` objects. Each (1 in this case) contains a list of `glm.FunctionDeclarations`, which describes a function and its arguments.

Using these classes could be necessary if the SDK fails to automatically create the `glm.FunctionDeclarations`s.

Here is a declaration for the same multiply function written using the `glm` classes.

Note that these classes just describe the function for the API, they don't include an implementation it. So using this doesn't work with automatic function calling, but functions don't always need an implementation.

In [ ]:
calculator = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='multiply',
        description="Returns the product of two numbers.",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'a':glm.Schema(type=glm.Type.NUMBER),
                'b':glm.Schema(type=glm.Type.NUMBER)
            },
            required=['a','b']
        )
      )
    ])

Give the model the calculator and ask it to do some arithmetic:

In [ ]:
model = genai.GenerativeModel('gemini-pro', tools=[calculator])
chat = model.start_chat()

response = chat.send_message(
    f"What's 234551 X 325552 ?",
)

Like before the model returns a `glm.FunctionCall` invoking the calculator's `multiply` function: 

In [ ]:
response.candidates

Execute the function yourself:

In [ ]:
fc = response.candidates[0].content.parts[0].function_call
assert fc.name == 'multiply'

result = fc.args['a'] * fc.args['b']
result

Send the result to the model, to continue the conversation:

In [ ]:
response = chat.send_message(
    glm.Content(
    parts=[glm.Part(
        function_response = glm.FunctionResponse(
          name='multiply',
          response={'result': result}
        )
    )]
  )
)

## Summary

Basic function calling is supported in the SDK. Remember that it is easier to manage using chat-mode, because of the natural back and forth structure. You're in charge of of actually calling the functions and sending results back to the model so it can produce a text-response. 